# Topic modelling for tweets of covid19

On this first exploratory work, we intend to find and visualize the main topics on a subset of our corpus of tweets about covid19. We perform un unsupervised learning in order to detect emerging topics for our study of the social narratives of the pandemic.

In [2]:
corpus_label = 'dhcovid_texts_2020-04-25_es_all' # We select all the tweets in spanish for april 25th.

with open(corpus_label+'.txt', 'r') as fi:
    tweets_spa_un_dia=fi.readlines()

In [3]:
with open('stopwords-spa.lst', 'r') as s: # Generic Spanish stopwords
    stop_words = s.read()
    
with open("stopwords-spa.extra.lst") as s: # This list is modified according to our corpus
    stop_words += s.read()

stop_words = stop_words.split()

Predictably, stopwords are of no interest for topic modelling, so we will filter them. And based on the results, we refresh the list of stopwords to eliminate noise (for ex, 'retwitt') or words that are too obvious (for ex, 'covid19').
Topic modelling focuses in nouns, since it is the category that prototypically holds the more information load of a text. We use spacy for part of speech detection but we disable the elements of the nlp pipeline that we don't use in order to reduce the processing time.

In [4]:
import spacy
from tqdm import tqdm

nlp = spacy.load('es_core_news_sm')

nlp_tweets_spa_un_dia = [nlp(tweet, disable=["parser", "ner", "textcat"]) for tweet in tqdm(tweets_spa_un_dia)]

100%|██████████| 50459/50459 [04:21<00:00, 193.21it/s]


In [5]:
from pprint import pprint
nouns_spa_un_dia = []

# We generate a list of tweets where each tweet is a list containing the nouns of the tweet only
for tweet_doc in tqdm(nlp_tweets_spa_un_dia):
    this_tweet_nouns = []
    
    for word in tweet_doc:
        if word.text not in stop_words and word.pos_=='NOUN':
            this_tweet_nouns.append(word.text)
            
    nouns_spa_un_dia.append(this_tweet_nouns)
        
for tweet_nouns in nouns_spa_un_dia[:15]:
    print("*", " ".join(tweet_nouns))
    

100%|██████████| 50459/50459 [00:07<00:00, 6600.39it/s]

* cifra casos coronavirus paso muertes
* bebe meses bombero
* exvicepresidente declaraciones uso desinfectante pacientes
* campaña trabajo personal linea combate comunidad
* primavera verano
* olvides medidas
* muerte reconfiguracion orden
* banqueros cuarentena coactivas embargos lugar infierno guayaquilenemergencia
* cifras pais compra urgencia ventiladores
* asistente
* lumbreras aqui
* aire ganas estupido enfermedad neta
* vecinos cabeza mierda beisbol mesa hija vecina urbanizacion total policia edificio hija vecina caso
* vivo par menopausia
* india ejemplo apertura comercios


In the previous step, we should also lemmatize in order to reduce the number of word forms. However, we observed that the lemmatization with spacy was not satisfactory for this corpus. We will try other lemmatizers in the future.
Other helpful processings that we plan to do here are finding bigrams and removing rare words and common words.

Now we will process our tweets with Gensim, a Python library for unsupervised topic modeling. 

In [ ]:
import gensim
from gensim import corpora

dictionary = corpora.Dictionary(nouns_spa_un_dia)
doc_term_matrix = [dictionary.doc2bow(tweet_nouns) for tweet_nouns in nouns_spa_un_dia]

We train our model with LDA for several number of topics and we save our models.

In [ ]:
from gensim.models.ldamodel import LdaModel

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

modelos_LDA = {}
topic_numbers_to_try = range(3, 15)

for n in topic_numbers_to_try:
    
    logging.basicConfig(filename=f"modelos_gensim/gensim.LDA_model_n{n:02}.log")
    
    print(f"Entrenando LDA model con {n} topics")
    modelo = LdaModel(
        doc_term_matrix,
        num_topics=n,
        id2word=dictionary,
        passes=20,
        eval_every=1,
        iterations=50
    )
    
    archivo_destino = f"modelos_gensim/modelo_gensim_ntopics{n:02}"
    print(f"Guardando en {archivo_destino}")
    modelo.save(archivo_destino)
    modelos_LDA[n] = modelo

Once the training is finished, we can use pyLDAvis to visualize the saved models:

In [63]:
from gensim.models.ldamodel import LdaModel
from gensim import corpora
import pyLDAvis.gensim
pyLDAvis.enable_notebook()


def cargar_modelo_LDA(ntopics):
    model_fp = f'modelos_gensim/modelo_gensim_ntopics{ntopics:02}'
    return LdaModel.load(model_fp)

def cargar_modelo_y_graficar(ntopics, tweets_procesados, verbose=False):
    '''
    ntopics: numero de topics
    tweets_procesados: lista de tweets que son listas de strings
    '''
    ldamodel = cargar_modelo_LDA(ntopics)
    
    if verbose:
        print(f"** Modelo con #topics = {ntopics} **")
        for i in range(ntopics):
            topic_words_and_weights = ldamodel.show_topic(i)
            topic_words = " ".join([word for word, weight in topic_words_and_weights])
            print(f" * ", topic_words)
        print()
    
    dictionary = corpora.Dictionary(tweets_procesados)
    doc_term_matrix = [dictionary.doc2bow(tweet_terms) for tweet_terms in tweets_procesados]
    return pyLDAvis.gensim.prepare(ldamodel, doc_term_matrix, dictionary)

Lets visualize the model for 10 topics:

In [31]:
cargar_modelo_y_graficar(10, nouns_spa_un_dia) 

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.276473 -0.026117       1        1  10.891291
6     -0.087548 -0.156417       2        1  10.778365
7     -0.177697  0.119699       3        1  10.646936
3     -0.098468  0.221917       4        1  10.570795
0     -0.011481 -0.142425       5        1  10.401235
1     -0.044801 -0.174901       6        1  10.067574
4      0.007252  0.151183       7        1   9.480697
9      0.259887  0.110791       8        1   9.391278
5     -0.114438  0.034807       9        1   9.175583
2     -0.009178 -0.138537      10        1   8.596252, topic_info=             Term         Freq        Total Category  logprob  loglift
0           casos  4180.000000  4180.000000  Default  30.0000  30.0000
351      personas  2200.000000  2200.000000  Default  29.0000  29.0000
158        crisis  1938.000000  1938.000000  Default  28.0000  28.0000
145          casa  1288.000000  1288.000000  Default  27.0000  27.0000
21        medidas  1319.000000  1319.000000  Default  26.0000  26.0000
..            ...          ...          ...      ...      ...      ...
337  trabajadores   220.285477   437.016144  Topic10  -4.4592   1.7688
121         salud   289.184570  2093.588623  Topic10  -4.1871   0.4743
125      pandemia   262.138397  2381.982178  Topic10  -4.2853   0.2470
75       acciones   130.047134   258.371399  Topic10  -4.9863   1.7673
178         parte   140.618408   686.857910  Topic10  -4.9081   0.8678

[429 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
1641      1  0.989272       1221
5530      1  0.992224      12872
299       9  0.994765       2020
3081      4  0.991397       2021
115       1  0.304983      abril
...     ...       ...        ...
820       2  0.994654  violencia
131       5  0.999042      virus
514       1  0.995592    warning
1736      8  0.991478       zona
850       7  0.994930      zonas

[465 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 7, 8, 4, 1, 2, 5, 10, 6, 3])

We can see that 10 topics it is not a good choice since there is a lot of topic overlapping as it is shown in the lower left area of the graphic. Lets try visualizing a lower number of topics:

In [57]:
cargar_modelo_y_graficar(4, nouns_spa_un_dia)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.054070  0.132173       1        1  27.435457
1     -0.232355 -0.231903       2        1  26.415487
2      0.311689 -0.131228       3        1  23.865244
0     -0.025263  0.230958       4        1  22.283806, topic_info=           Term         Freq        Total Category  logprob  loglift
0         casos  3834.000000  3834.000000  Default  30.0000  30.0000
11    pacientes  1440.000000  1440.000000  Default  29.0000  29.0000
145        casa  1201.000000  1201.000000  Default  28.0000  28.0000
125    pandemia  2237.000000  2237.000000  Default  27.0000  27.0000
131       virus  1024.000000  1024.000000  Default  26.0000  26.0000
..          ...          ...          ...      ...      ...      ...
21      medidas   281.108551  1254.791504   Topic4  -5.1680   0.0053
115       abril   268.886505  1103.637451   Topic4  -5.2124   0.0892
67        mundo   264.435059  1122.010620   Topic4  -5.2291   0.0560
121       salud   255.935898  1901.387695   Topic4  -5.2618  -0.5041
102  emergencia   221.478287   721.484924   Topic4  -5.4064   0.3203

[208 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
115       1  0.139539     abril
115       2  0.486573     abril
115       3  0.129572     abril
115       4  0.243739     abril
75        1  0.996061  acciones
...     ...       ...       ...
74        3  0.998527  victimas
304       3  0.999123      vida
676       4  0.996780     vidas
1414      2  0.999131   viernes
131       4  0.999052     virus

[211 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 3, 1])

There is still some topic overlapping. Lets try 5 topics:

In [58]:
cargar_modelo_y_graficar(5, nouns_spa_un_dia)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.295283  0.071983       1        1  21.295908
3     -0.065692 -0.277441       2        1  20.759830
0     -0.209394  0.005005       3        1  19.863430
4     -0.128557  0.253904       4        1  19.195440
1      0.108360 -0.053451       5        1  18.885397, topic_info=         Term         Freq        Total Category  logprob  loglift
0       casos  3827.000000  3827.000000  Default  30.0000  30.0000
121     salud  1948.000000  1948.000000  Default  29.0000  29.0000
35       pais  1261.000000  1261.000000  Default  28.0000  28.0000
158    crisis  1888.000000  1888.000000  Default  27.0000  27.0000
3     muertes  1094.000000  1094.000000  Default  26.0000  26.0000
..        ...          ...          ...      ...      ...      ...
125  pandemia   408.325714  2233.693115   Topic5  -4.6292  -0.0326
147       dia   280.823059   936.427368   Topic5  -5.0035   0.4624
221       fin   238.378036   523.679749   Topic5  -5.1674   0.8798
609      mayo   218.825211   382.474304   Topic5  -5.2530   1.1084
21    medidas   250.464035  1259.126465   Topic5  -5.1179   0.0519

[263 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
299       2  0.995839         2020
115       1  0.564291        abril
115       2  0.158002        abril
115       4  0.277180        abril
797       4  0.997247  actividades
...     ...       ...          ...
131       1  0.267397        virus
131       4  0.217559        virus
131       5  0.514667        virus
514       1  0.991328      warning
850       3  0.997884        zonas

[293 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 1, 5, 2])

This is a good number of topics. However, in the future we should calculate topic coherence of each topic so we can obtain the best number of topics automatically.

We can save the graphic of the best topic number for our corpus, the 5 topic model, to share it in other applications, like our project website:

In [70]:
pyLDAvis.save_html(cargar_modelo_y_graficar(5, nouns_spa_un_dia), 'modelos_gensim/modelo05.token-vis.html')

We can try other visualizations, like a cicle pack plot, that gives us a good idea of the main words of each topic at a glance:

In [69]:
import pandas as pd


def hacer_csv_para_rawgraphsio(modelo, out_fp, top_n=10):
    records = []
    
    for i in range(modelo.num_topics):
        for word, weight in modelo.show_topic(i, topn=top_n):
            records.append((i, word, weight))

    df = pd.DataFrame(records, columns=["topic_id", "word", "weight"])
    df = df.groupby("topic_id").head(top_n).reset_index(drop=True)
    print(f"Escribiendo CSV en: {out_fp}")
    df.to_csv(out_fp, index=False)
    

modelo = cargar_modelo_LDA(ntopics=5)
fp = "modelos_gensim/modelo05.token.csv"
hacer_csv_para_rawgraphsio(modelo, fp)

Escribiendo CSV en: modelos_gensim/modelo05.token.csv


<img src="modelo.05.tokens.png" />